# 03b - Streaming Gold & Agrégations Temps Réel

Pipeline d'agrégation temps réel :
- **Silver → Gold** : Agrégations avec fenêtres temporelles (Tumbling & Sliding Windows)
- **2 Requêtes métier imposées :**
  1. **Tumbling Window** : Trafic par pays (fenêtre fixe 30s)
  2. **Sliding Window** : Vitesse/Altitude moyenne par catégorie (fenêtre 1m, glissement 10s)

## Configuration & Imports

In [1]:
from pyspark.sql.functions import (
    col, window, count, avg, max as spark_max, min as spark_min,
    sum as spark_sum, stddev, round, desc
)
from config import get_s3_path, create_spark_session
import time
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max as spark_max
import os

# Chemins S3
SILVER_PATH = get_s3_path("silver", "flights")
GOLD_TRAFFIC_PATH = get_s3_path("gold", "traffic_by_country")
GOLD_METRICS_PATH = get_s3_path("gold", "metrics_by_category")
CHECKPOINT_TRAFFIC = get_s3_path("checkpoints", "gold_traffic_by_country")
CHECKPOINT_METRICS = get_s3_path("checkpoints", "gold_metrics_by_category")

spark = create_spark_session("StreamingGold")

print(f"✅ Input Silver:        {SILVER_PATH}")
print(f"✅ Gold Traffic:        {GOLD_TRAFFIC_PATH}")
print(f"✅ Gold Metrics:        {GOLD_METRICS_PATH}")

✅ Configuration chargée depuis .env
:: loading settings :: url = jar:file:/opt/conda/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.spark#spark-hadoop-cloud_2.12 added as a dependency
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0e2213de-7443-4108-8680-476fc8d9b270;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.apache.spark#spark-hadoop-cloud_2.12;3.5.3 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.c

✅ Spark Session 'StreamingGold' configurée
✅ Input Silver:        s3a://datalake/silver/flights
✅ Gold Traffic:        s3a://datalake/gold/traffic_by_country
✅ Gold Metrics:        s3a://datalake/gold/metrics_by_category


## Requête 1 : Tumbling Window - Trafic par Pays

**Objectif :** Compter les avions uniques par pays d'origine sur des fenêtres fixes de 30 secondes.

**Cas d'usage métier :** Monitoring du trafic par pays en temps réel pour détecter les pics de congestion.

**Window Type :** Tumbling (fenêtres qui ne se chevauchent pas)

In [ ]:
df_silver_stream = spark.readStream.format("delta").load(SILVER_PATH)

# Requête 1: Tumbling Window (30s) - Trafic par pays
query_traffic = df_silver_stream \
    .filter(col("origin_country").isNotNull()) \
    .groupBy(
        window(col("event_timestamp"), "30 seconds"),
        col("origin_country")
    ) \
    .agg(
        count("icao24").alias("flight_count")
    )

print("🚀 Requête 1: Tumbling Window (30s) - Trafic par Pays")
print("   Structure: [window_start, window_end, origin_country, flight_count]")

query1 = query_traffic \
    .writeStream \
    .format("delta") \
    .outputMode("complete") \
    .option("checkpointLocation", CHECKPOINT_TRAFFIC) \
    .option("mergeSchema", "true") \
    .start(GOLD_TRAFFIC_PATH)

print(f"✅ Query1 ID: {query1.id}")
print(f"   Destination: {GOLD_TRAFFIC_PATH}")

26/01/22 20:21:04 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


🚀 Requête 1: Tumbling Window (30s) - Trafic par Pays
   Structure: [window_start, window_end, origin_country, flight_count]


26/01/22 20:21:13 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


✅ Query1 ID: 42b870ab-898d-41bb-a5ec-c1f8392bc9dc
   Destination: s3a://datalake/gold/traffic_by_country


26/01/22 20:21:16 ERROR NonFateSharingFuture: Failed to get result from future
scala.runtime.NonLocalReturnControl
26/01/22 20:21:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## Requête 2 : Sliding Window - Vitesse & Altitude par Catégorie

**Objectif :** Calculer la vitesse moyenne et l'altitude moyenne par catégorie d'avion sur une fenêtre glissante.

**Fenêtre :** 1 minute de durée, glissant toutes les 10 secondes.

**Cas d'usage métier :** Analyser les profils de vol par catégorie (Light vs Heavy) pour calibrer les modèles ML.

**Window Type :** Sliding (fenêtres qui se chevauchent)

In [ ]:
# Requête 2: Sliding Window (1m duration, 10s slide) - Métriques par Catégorie
query_metrics = df_silver_stream \
    .filter(col("category").isNotNull()) \
    .filter(col("velocity_kmh") > 0) \
    .filter(col("altitude_meters").isNotNull()) \
    .groupBy(
        window(col("event_timestamp"), "1 minute", "10 seconds"),
        col("category")
    ) \
    .agg(
        count("icao24").alias("aircraft_count"),
        round(avg(col("velocity_kmh")), 2).alias("avg_velocity_kmh"),
        round(spark_max(col("velocity_kmh")), 2).alias("max_velocity_kmh"),
        round(spark_min(col("velocity_kmh")), 2).alias("min_velocity_kmh"),
        round(stddev(col("velocity_kmh")), 2).alias("stddev_velocity_kmh"),
        round(avg(col("altitude_meters")), 0).alias("avg_altitude_m"),
        round(spark_max(col("altitude_meters")), 0).alias("max_altitude_m"),
        round(spark_min(col("altitude_meters")), 0).alias("min_altitude_m")
    )

print("🚀 Requête 2: Sliding Window (1m duration, 10s slide) - Vitesse/Altitude par Catégorie")
print("   Structure: [window_start, window_end, category, aircraft_count, avg_velocity, ...]")

query2 = query_metrics \
    .writeStream \
    .format("delta") \
    .outputMode("complete") \
    .option("checkpointLocation", CHECKPOINT_METRICS) \
    .option("mergeSchema", "true") \
    .start(GOLD_METRICS_PATH)

print(f"✅ Query2 ID: {query2.id}")
print(f"   Destination: {GOLD_METRICS_PATH}")

🚀 Requête 2: Sliding Window (1m duration, 10s slide) - Vitesse/Altitude par Catégorie
   Structure: [window_start, window_end, category, aircraft_count, avg_velocity, ...]


26/01/22 20:21:20 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


✅ Query2 ID: 940df0b5-0c6e-414c-a994-2cc8104942f6
   Destination: s3a://datalake/gold/metrics_by_category


26/01/22 20:21:23 ERROR NonFateSharingFuture: Failed to get result from future
scala.runtime.NonLocalReturnControl
26/01/22 20:21:23 WARN HDFSBackedStateStoreProvider: The state for version 89 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
26/01/22 20:21:23 WARN HDFSBackedStateStoreProvider: The state for version 89 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
26/01/22 20:21:23 WARN HDFSBackedStateStoreProvider: The state for version 89 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
26/01/22 20:21:23 WARN HDFSBackedStateStoreProvider: The state for version 89 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
2

## Monitoring des Streams

In [4]:
import time

print("📊 Monitoring des streams (Ctrl+C pour arrêter)")
print("="*70)

try:
    while True:
        print(f"\n⏱️  {time.strftime('%H:%M:%S')}")
        print(f"  Query 1 (Traffic):  {query1.status}")
        print(f"  Query 2 (Metrics):  {query2.status}")
        time.sleep(30)
except KeyboardInterrupt:
    print("\n⏹️  Arrêt demandé...")

📊 Monitoring des streams (Ctrl+C pour arrêter)

⏱️  19:51:49
  Query 1 (Traffic):  {'message': 'Initializing sources', 'isDataAvailable': False, 'isTriggerActive': True}
  Query 2 (Metrics):  {'message': 'Initializing sources', 'isDataAvailable': False, 'isTriggerActive': False}


26/01/22 19:51:51 ERROR NonFateSharingFuture: Failed to get result from future
scala.runtime.NonLocalReturnControl
26/01/22 19:51:51 ERROR NonFateSharingFuture: Failed to get result from future
scala.runtime.NonLocalReturnControl
26/01/22 19:51:52 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.



⏹️  Arrêt demandé...


26/01/22 19:51:57 WARN HDFSBackedStateStoreProvider: The state for version 53 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
26/01/22 19:51:57 WARN HDFSBackedStateStoreProvider: The state for version 53 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
26/01/22 19:51:57 WARN HDFSBackedStateStoreProvider: The state for version 52 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
26/01/22 19:51:57 WARN HDFSBackedStateStoreProvider: The state for version 52 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
26/01/22 19:51:57 WARN HDFSBackedStateStoreProvider: The state for version 52 doesn't exist in loadedMaps. Reading s

## Arrêt des Streams

In [ ]:
query1.stop()
query2.stop()
print("✅ Tous les streams Gold arrêtés")

✅ Tous les streams Gold arrêtés


## Vérification des Résultats

In [ ]:
# Lire les derniers agrégats pour vérifier
print("📊 Vérification du Gold Traffic (3 dernières fenêtres):\n")
spark.read.format("delta").load(GOLD_TRAFFIC_PATH) \
    .orderBy(desc("window")) \
    .limit(15) \
    .show(truncate=False)

print("\n📊 Vérification du Gold Metrics (3 dernières fenêtres):\n")
spark.read.format("delta").load(GOLD_METRICS_PATH) \
    .orderBy(desc("window")) \
    .limit(15) \
    .show(truncate=False)

📊 Vérification du Gold Traffic (3 dernières fenêtres):



26/01/22 18:04:07 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 